# OpenStreetMap Data Case Study MongoDB

## Map Area
Grand Rapids, Michigan, United States
https://www.openstreetmap.org/export#map=13/42.9662/-85.6492 

In [33]:
#Allows us to connect to MongoDB
!pip install pymongo 

## Uploading the data

In [28]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint
import pymongo
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client['test']

osm_file = open ("map.json", "r")

#Set up street type dictionary to allow us to sift through street names and pull out abnormalities. 
#This will help us clean the data.
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE) #pulls out last word in street name
street_types = defaultdict(set) #keeps track of unusual streets

expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Lane", "Parkway", "Road", "Trail", "Commons"]

<_io.TextIOWrapper name='map.json' mode='r' encoding='cp1252'>


## Auditing the Data

In [29]:
#This will take the end of a street name to get the type of street, and then set it in defaultdict if it is not expected. 
def audit_street_type(street_types, street_name):
    m = street_type_re.search(Street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

In [35]:
#This allows us to sort the dictionary and print it to help us explore the abnormal street types.
def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print ("%s: %d" %(k, v))

In [36]:
#This looks to see if an element is a street name.
def is_street_name(elem):
    return(elem.attrib['k'] == "addr:street") #checks for k attribute, and changes it to the new value.

In [38]:
#When it sees a start tag, generates next item in iteration using tuple
def audit(osmfile):
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "way": #If way, iter to return tags nested within way
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    pprint.pprint(dict(street_types)) #prints data from audit
    return(street_types)

## Problems Encountered During Audit

defaultdict(<class 'set'>, {})


## Data Overview and Additional Suggestions


## Conclusion